In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite python-dotenv

In [11]:
from dotenv import load_dotenv
import json

# Load the environment variables from the .env file
load_dotenv()

True

# Set Up LangSmith

In [ ]:
%pip install -U langchain langchain-openai

The following environment variables are being used in the background:

```bash
OPENAI_API_KEY=<api-key-from-openai>

# LangSmith
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=<api-key-from-langsmith>
LANGCHAIN_PROJECT=<project-name>
```

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-84c7f2b4-667a-41f1-a5fc-9d8b8d2a4bea-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

# Define Tools

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(json.dumps(search_results, indent=4))
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730422217, 'localtime': '2024-10-31 17:50'}, 'current': {'last_updated_epoch': 1730421900, 'last_updated': '2024-10-31 17:45', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 16.8, 'wind_kph': 27.0, 'wind_degree': 203, 'wind_dir': 'SSW', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 65, 'cloud': 50, 'feelslike_c': 16.7, 'feelslike_f': 62.1, 'windchill_c': 14.2, 'windchill_f': 57.5, 'heatindex_c': 15.1, 'heatindex_f': 59.2, 'dewpoint_c': 9.0, 'dewpoint_f': 48.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 23.6, 'gust_kph': 38.1}}"}, {'url': 'https://weatherspark.com/h/m/557

# Using LLMs

In [13]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4")

In [ ]:
response = model.invoke([HumanMessage(content="hi!")])
response.content

Enable model to do tool calling of tools defined in the previous section.

In [15]:
model_with_tools = model.bind_tools(tools)

In [18]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [19]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_G58ImaVFbxpsazbrxxAK5yoK', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

# Create the Agent

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [20]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the Agent

Without tool call

In [21]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='895e1bed-bbfd-4114-ba77-b34cdf533bde'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3c1d6f3a-ff20-4eb2-9244-cc542c29a717-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]

With tool call

In [22]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='6c204a51-57e3-4c53-b643-5ef1fe94ec9c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aLCdgpStSBZE3diDMXa4z1hM', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b4dc3593-c1c8-4575-8513-6293c61416de-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_aLCdgpStSBZE3diDMXa4z1hM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88

# Stream Tokens

In [23]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Orlando?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Orlando'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Orlando\', \'region\': \'Florida\', \'country\': \'United States of America\', \'lat\': 28.5381, \'lon\': -81.3794, \'tz_id\': \'America/New_York\', \'localtime_epoch\': 1730424950, \'localtime\': \'2024-10-31 21:35\'}, \'current\': {\'last_updated_epoch\': 1730424600, \'last_updated\': \'2024-10-31 21:30\', \'temp_c\': 23.3, \'temp_f\': 73.9, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 7.8, \'wind_kph\': 12.6, \'wind_degree\': 72, \'wind_dir\': \'ENE\', \'pressure_mb\': 1021.0, \'pressure_in\': 30.15, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 79, \'cloud\': 75, \'feelslike_c\': 25.2, \'feelslike_f\': 77.4, \'windchill_c\': 22.

# LLM Memory (skipped)

See [LangChain docs](https://python.langchain.com/docs/tutorials/agents/#adding-in-memory) for adding memory.